In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import Library

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, AveragePooling2D, Input, BatchNormalization, MaxPooling2D, Activation, Flatten, Dense, Dropout
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing import image
import scipy
import os
import cv2

# Load Data
용량 문제로 github 업로드 불가, 데이터 다운로드 필요

아래 링크 접속하여 fer2013.csv 다운로드

[fer2013.csv](https://www.kaggle.com/datasets/nicolejyt/facialexpressionrecognition)

In [ ]:
PATH = '/content/drive/MyDrive/졸프/ML/data'

In [ ]:
df = pd.read_csv(PATH+'fer2013.csv')

# Preprocess Data

In [ ]:
def preprocess_pixels(pixel_data):
    images = []
    for i in range(len(pixel_data)):
        img = np.fromstring(pixel_data[i], dtype='int', sep=' ')
        img = img.reshape(48,48,1)
        images.append(img)

    X = np.array(images)

    return X

In [ ]:
X = preprocess_pixels(df['pixels'])
label_data = df['emotion']

In [ ]:
from sklearn.utils import shuffle

X = preprocess_pixels(df['pixels'])
X_flat = X.reshape(X.shape[0], -1)
X_flat, label_data = shuffle(X_flat, label_data)

smote = SMOTE(random_state=0)
X_flat_over, Y_over = smote.fit_resample(X_flat, label_data)

X_over = X_flat_over.reshape(-1, 48, 48, 1)
X = X_over
Y = Y_over

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, stratify=Y, test_size=0.2, random_state=45)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, stratify=Y_train, test_size=0.2, random_state=45)

In [ ]:
X_aug = ImageDataGenerator(
    featurewise_center=False,
    featurewise_std_normalization=False,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=.1,
    horizontal_flip=True)

X_aug.fit(X_train)

# Data Encoding

In [ ]:
encoder = OneHotEncoder()
encoder.fit(Y_train.values.reshape(-1, 1))
Y_train = encoder.transform(Y_train.values.reshape(-1, 1)).toarray()
Y_test = encoder.transform(Y_test.values.reshape(-1, 1)).toarray()
Y_val = encoder.transform(Y_val.values.reshape(-1, 1)).toarray()

In [ ]:
es = EarlyStopping(patience=60)
mc = ModelCheckpoint(PATH, monitor='val_loss', save_best_only=True)
rlr = ReduceLROnPlateau(factor=0.1, patience=5)

# Make CNN model & Training

In [ ]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3,3), strides=(1,1), padding='same', input_shape=(48,48,1)))
model.add(BatchNormalization(axis=3))
model.add(Activation('relu'))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(64, (3,3), strides=(1,1), padding='same', kernel_regularizer=tf.keras.regularizers.l2(0.0005)))
model.add(BatchNormalization(axis=3))
model.add(Activation('relu'))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(64, (3,3), strides=(1,1), padding='same', kernel_regularizer=tf.keras.regularizers.l2(0.0005)))
model.add(BatchNormalization(axis=3))
model.add(Activation('relu'))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(128, (3,3), strides=(1,1), padding='same', kernel_regularizer=tf.keras.regularizers.l2(0.0005)))
model.add(BatchNormalization(axis=3))
model.add(Activation('relu'))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(128, (3,3), strides=(1,1), padding='same', kernel_regularizer=tf.keras.regularizers.l2(0.0005)))
model.add(BatchNormalization(axis=3))
model.add(Activation('relu'))
model.add(MaxPooling2D((2,2)))
model.add(Flatten())

model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(7, activation='softmax'))

adam = Adam(lr=0.0001)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
history = model.fit(X_aug.flow(X_train, Y_train), epochs=100, validation_data=(X_val, Y_val), callbacks=[es, mc, rlr])


Epoch 1/100
1259/1259 [==============================] - 41s 25ms/step - loss: 1.8299 - accuracy: 0.3591 - val_loss: 1.8496 - val_accuracy: 0.3534 - lr: 0.0010
Epoch 2/100
1259/1259 [==============================] - 30s 24ms/step - loss: 1.5203 - accuracy: 0.4892 - val_loss: 1.7425 - val_accuracy: 0.3603 - lr: 0.0010
Epoch 3/100
1259/1259 [==============================] - 29s 23ms/step - loss: 1.4285 - accuracy: 0.5231 - val_loss: 1.4926 - val_accuracy: 0.4940 - lr: 0.0010
Epoch 4/100
1259/1259 [==============================] - 27s 22ms/step - loss: 1.3853 - accuracy: 0.5454 - val_loss: 1.4995 - val_accuracy: 0.5281 - lr: 0.0010
Epoch 5/100
1259/1259 [==============================] - 29s 23ms/step - loss: 1.3656 - accuracy: 0.5550 - val_loss: 1.3427 - val_accuracy: 0.5715 - lr: 0.0010
Epoch 6/100
1259/1259 [==============================] - 30s 24ms/step - loss: 1.3314 - accuracy: 0.5696 - val_loss: 1.2854 - val_accuracy: 0.5897 - lr: 0.0010
Epoch 7/100
1259/1259 [=================

# Save model

In [ ]:
model.save("fer_model.h5")
score = model.evaluate(X_val, Y_val, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1]*100)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


315/315 [==============================] - 2s 5ms/step - loss: 0.8835 - accuracy: 0.7177
Test loss: 0.883521556854248
Test accuracy: 71.77194952964783
